#### 1. 필요한 라이브러리 및 패키지 임포트

In [6]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium .webdriver.chrome.service import Service

In [7]:
import pandas as pd
import numpy as np

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import csv
from bs4 import BeautifulSoup as bs
import requests
import math

import re
import urllib.request
import json
import datetime

import random

import chardet

#### 2. 데이터 불러오기

In [8]:
bizno_df=pd.read_csv('bizno_df.csv')
bizno_df

,업체명,사업자등록번호,참여횟수,낙찰횟수,낙찰률(%),가중 낙찰률,성과 정규화 지수,상대적 성과 지수
0,종합건축사사무소한알건축,3030275710,1,1,100.0,0.693,1.0,86.311776
1,주식회사 세미종합건축사사무소,3128668566,1,1,100.0,0.693,1.0,86.311776
2,타코 건축사사무소,7822800877,1,1,100.0,0.693,1.0,86.311776
3,(주)대오엔지니어링,2058131280,1,1,100.0,0.693,1.0,86.311776
4,비원건축사사무소 주식회사,4628102369,1,1,100.0,0.693,1.0,86.311776
...,...,...,...,...,...,...,...,...
1047,모아건축사사무소,1160724233,2,0,0.0,0.000,0.0,0.000000
1048,모음 건축사사무소,5200302352,1,0,0.0,0.000,0.0,0.000000
1049,무이건축사사무소,3485000168,1,0,0.0,0.000,0.0,0.000000
1050,미가건축사사무소,7480901410,1,0,0.0,0.000,0.0,0.000000


#### 3. 웹브라우저 열어서 로그인

In [ ]:
url = 'https://www.nicebizline.com'

# 헤더 변경으로 차단 우회
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers",
    "DNT": "1",
}

driver = webdriver.Chrome()
driver.get(url)
time.sleep(random.randint(4, 10)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

#### 4. 기업 정보 크롤링

In [13]:
def address_crawler(winrate_df, start_index):
    num_list = winrate_df['사업자등록번호'].tolist()

    address_list = [] # 주소
    service_type_list = [] # 산업형태
    company_num_list = [] # 사업자등록번호 

    try:
        for num in num_list[start_index:]:
            # 검색어 입력
            search_box = driver.find_element(By.ID, "search") # 검색박스 
            search_box.clear() # 이전 검색어 삭제(기업조회 이후에 검색박스가 자동으로 초기화 안되는 구조)
            search_box.send_keys(num) # 검색박스에 해당 기업의 사업자 등록번호 입력
            search_box.send_keys(Keys.RETURN) 
            time.sleep(random.randint(1, 3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

            try:
                # 탭에서 기업검색을 클릭하는 동작 생성
                tabs = driver.find_elements(By.CSS_SELECTOR, 'h4.v-tab')
                for tab in tabs:
                    if tab.find_element(By.CSS_SELECTOR, 'span').text == '기업 검색':
                        tab.click()
                time.sleep(random.randint(1, 3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

                # 기업의 주소, 산업형태 데이터 추출
                soup = bs(driver.page_source, 'lxml')
                address = soup.find('td', {'class': 'text-start td--fixedw152'}).text # 주소 
                service_type = soup.find('td', {'class': 'text-start td--fixedw208'}).text # 산업형태
                time.sleep(random.randint(1, 3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

            # 사업자등록번호 조회결과 상세 정보가 없다면 결측치로 처리
            except Exception as e: # 예외처리
                address = np.nan
                service_type = np.nan

            # 탭 닫기 (탭이 10개 이상이 되면 더이상의 검색작동이 안되는 구조)
            driver.find_element(By.CSS_SELECTOR, 'button.nbl__common--tab-card-close').click() 
            driver.find_element(By.CSS_SELECTOR, '#app > div.v-dialog__content.v-dialog__content--active > div > div > div.v-card__actions.popup__footer > button.popup__footer__btn.confirm__btn.v-btn.v-btn--is-elevated.v-btn--has-bg.v-btn--tile.theme--light.v-size--default.primary > span').click()

            # 리스트에 데이터 저장    
            company_num_list.append(num)
            address_list.append(address)
            service_type_list.append(service_type)


    # 메모리 초과 등 예상치 못하게 중단되었을 때 예외처리
    except Exception as e:
        print("Out Of Memory")

    # 데이터프레임 생성
    data = {
    "사업자등록번호": company_num_list,
    "주소": address_list,
    "사업형태": service_type_list
    }

    return pd.DataFrame(data) # 데이터 반환

In [16]:
address_crawler(bizno_df,0) 

,사업자등록번호,주소,사업형태
0,3030275710,충북 충주시 금제9길 18-1,건축설계 및 관련 서비스업
1,3128668566,충남 천안시 서북구 불당9길 46-1,건축설계 및 관련 서비스업
2,7822800877,NaN,NaN
3,2058131280,"서울 동대문구 망우로 67, 206호",건물 및 토목 엔지니어링 서비스업
4,4628102369,"경남 창녕군 창녕읍 문화공원길 21, 3층",건축설계 및 관련 서비스업
